# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pevek,69.7008,170.3133,4.05,87,100,1.03,RU,1649831944
1,1,san cristobal,7.7669,-72.2250,68.11,74,100,2.37,VE,1649831944
2,2,macamic,48.7502,-78.9996,32.88,88,100,1.99,CA,1649831944
3,3,ilulissat,69.2167,-51.1000,21.22,93,75,11.50,GL,1649831944
4,4,mar del plata,-38.0023,-57.5575,51.10,87,2,11.99,AR,1649831944


In [3]:
len(city_data_df)

563

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create a list of locations that are between 70 and 80 deg with no cloud cover and wind speed less than 10 mph. drop non
# qualifiying locations and create a perfect_place list
perfect_place = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
perfect_place


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,busselton,-33.6500,115.3333,74.50,39,0,9.53,AU,1649831946
40,40,mount gambier,-37.8333,140.7667,73.31,35,0,8.05,AU,1649831947
108,108,port elizabeth,-33.9180,25.5701,72.32,80,0,2.30,ZA,1649831953
351,351,maceio,-9.6658,-35.7353,72.84,94,0,3.44,BR,1649831866
555,555,chipinge,-20.1883,32.6236,72.75,69,0,2.30,ZW,1649831996


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = perfect_place[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
32,busselton,AU,-33.6500,115.3333,
40,mount gambier,AU,-37.8333,140.7667,
108,port elizabeth,ZA,-33.9180,25.5701,
351,maceio,BR,-9.6658,-35.7353,
555,chipinge,ZW,-20.1883,32.6236,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
32,busselton,AU,-33.6500,115.3333,Observatory Guest House
40,mount gambier,AU,-37.8333,140.7667,The Commodore
108,port elizabeth,ZA,-33.9180,25.5701,39 On Nile Guest House
351,maceio,BR,-9.6658,-35.7353,Hotel Ponta Verde Maceió
555,chipinge,ZW,-20.1883,32.6236,Chipinge Breeze Lodge


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))